In [1]:
# get the data set
from word_mapping import word_mapping
from tag_mapping import tag_mapping
from utils import *
from check import check

cn_train_data, cn_valid_data = get_data_set('Chinese')
cn_TagMapping = tag_mapping('Chinese')
cn_WordMapping = word_mapping(cn_train_data)
cn_train_data.get_tag_mapping(cn_TagMapping.encode_mapping)
cn_train_data.get_word_mapping(cn_WordMapping.encode_mapping)
cn_valid_data.get_tag_mapping(cn_TagMapping.encode_mapping)
cn_valid_data.get_word_mapping(cn_WordMapping.encode_mapping)

en_train_data, en_valid_data = get_data_set('English')
en_TagMapping = tag_mapping('English')
en_WordMapping = word_mapping(en_train_data)
en_train_data.get_tag_mapping(en_TagMapping.encode_mapping)
en_train_data.get_word_mapping(en_WordMapping.encode_mapping)
en_valid_data.get_tag_mapping(en_TagMapping.encode_mapping)
en_valid_data.get_word_mapping(en_WordMapping.encode_mapping)

In [2]:
""" # the training of HMM

from HMM import *

# Chinese version
cn_HMM_path = 'HMM_model/HMM_Chinese.npz'
cn_model = HMM(cn_TagMapping.num_tag, cn_WordMapping.num_word)
smoothing_factor = 1
cn_model.param_estimate(cn_train_data, smoothing_factor=smoothing_factor)
cn_model.save(cn_HMM_path)

# English version
en_HMM_path = 'HMM_model/HMM_English.npz'
en_model = HMM(en_TagMapping.num_tag, en_WordMapping.num_word)
smoothing_factor = 1
en_model.param_estimate(en_train_data, smoothing_factor=smoothing_factor)
en_model.save(en_HMM_path) """

" # the training of HMM\n\nfrom HMM import *\n\n# Chinese version\ncn_HMM_path = 'HMM_model/HMM_Chinese.npz'\ncn_model = HMM(cn_TagMapping.num_tag, cn_WordMapping.num_word)\nsmoothing_factor = 1\ncn_model.param_estimate(cn_train_data, smoothing_factor=smoothing_factor)\ncn_model.save(cn_HMM_path)\n\n# English version\nen_HMM_path = 'HMM_model/HMM_English.npz'\nen_model = HMM(en_TagMapping.num_tag, en_WordMapping.num_word)\nsmoothing_factor = 1\nen_model.param_estimate(en_train_data, smoothing_factor=smoothing_factor)\nen_model.save(en_HMM_path) "

In [3]:
# the training of my handmade CRF
from my_handmade_CRF import feature_functions, CRF
import pickle
from tqdm import tqdm

""" # Chinese version
cn_features = feature_functions(cn_train_data)
cn_model = CRF(cn_features, cn_TagMapping.num_tag)

cn_model.train(cn_train_data, max_epoch=15)

with open('handmade_CRF_model/CRF_Chinese.pkl', 'wb') as f:
    pickle.dump(cn_model, f)

#English version
en_features = feature_functions(en_train_data)
en_model = CRF(en_features, en_TagMapping.num_tag)

en_model.train(en_train_data, max_epoch=15)

with open('handmade_CRF_model/CRF_English.pkl', 'wb') as f:
    pickle.dump(en_model, f)
 """

" # Chinese version\ncn_features = feature_functions(cn_train_data)\ncn_model = CRF(cn_features, cn_TagMapping.num_tag)\n\ncn_model.train(cn_train_data, max_epoch=15)\n\nwith open('handmade_CRF_model/CRF_Chinese.pkl', 'wb') as f:\n    pickle.dump(cn_model, f)\n\n#English version\nen_features = feature_functions(en_train_data)\nen_model = CRF(en_features, en_TagMapping.num_tag)\n\nen_model.train(en_train_data, max_epoch=15)\n\nwith open('handmade_CRF_model/CRF_English.pkl', 'wb') as f:\n    pickle.dump(en_model, f)\n "

In [4]:
# the training of BiLSTM CRF
import torch
import torch.nn as nn
import BiLSTM_CRF

dropout = 0.5
embed_size = 256
hidden_size = 256
batch_size = 32
max_epoch = 20
lr = 0.001
clip_max_norm = 5.0




In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# English version
en_tag_size = en_TagMapping.num_tag
en_vocab_size = en_WordMapping.num_word

en_model = BiLSTM_CRF.BiLSTMCRF(en_tag_size, en_vocab_size, dropout, embed_size, hidden_size).to(device)
for name, param in en_model.named_parameters():
    if 'weight' in name:
        nn.init.normal_(param.data, 0, 0.01)
    else:
        nn.init.constant_(param.data, 0)
optimizer = torch.optim.Adam(en_model.parameters(), lr=lr)
for epoch in range(max_epoch):
    num_iter = 0
    for sentences, tags in tqdm(batch_iter(en_train_data, batch_size=batch_size)):
        num_iter += 1
        sentences, sent_lengths = pad(sentences, en_vocab_size - 1, device)
        tags, _ = pad(tags, en_tag_size - 1, device)

        optimizer.zero_grad()
        batch_loss = en_model(sentences, tags, sent_lengths)
        loss = batch_loss.mean()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(en_model.parameters(), max_norm=clip_max_norm)
        optimizer.step()

    print('Epoch: %d, Iter: %d, Loss: %.4f' % (epoch, num_iter, loss.item()))

en_model_save_path = 'BiLSTM_CRF/English.pth'
en_model.save(en_model_save_path)

439it [00:16, 25.92it/s]


Epoch: 0, Iter: 439, Loss: 10.4419


439it [00:17, 25.78it/s]


Epoch: 1, Iter: 439, Loss: 5.3788


439it [00:17, 25.80it/s]


...

Epoch: 17, Iter: 439, Loss: 2.4293


439it [00:16, 26.26it/s]


Epoch: 18, Iter: 439, Loss: 1.8231


439it [00:16, 26.06it/s]

Epoch: 19, Iter: 439, Loss: 3.1246


In [6]:
""" cn_tag_size = cn_TagMapping.num_tag
cn_vocab_size = cn_WordMapping.num_word

cn_model = BiLSTM_CRF.BiLSTMCRF(cn_tag_size, cn_vocab_size, dropout, embed_size, hidden_size).to(device)
for name, param in cn_model.named_parameters():
    if 'weight' in name:
        nn.init.normal_(param.data, 0, 0.01)
    else:
        nn.init.constant_(param.data, 0)
optimizer = torch.optim.Adam(cn_model.parameters(), lr=lr)
for epoch in range(max_epoch):
    num_iter = 0
    for sentences, tags in tqdm(batch_iter(cn_train_data, batch_size=batch_size)):
        num_iter += 1
        sentences, sent_lengths = pad(sentences, cn_vocab_size - 1, device)
        tags, _ = pad(tags, cn_tag_size - 1, device)

        optimizer.zero_grad()
        batch_loss = cn_model(sentences, tags, sent_lengths)
        loss = batch_loss.mean()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(cn_model.parameters(), max_norm=clip_max_norm)
        optimizer.step()

    print('Epoch: %d, Iter: %d, Loss: %.4f' % (epoch, num_iter, loss.item()))

cn_model_save_path = 'BiLSTM_CRF/Chinese.pth'
cn_model.save(cn_model_save_path) """

" cn_tag_size = cn_TagMapping.num_tag\ncn_vocab_size = cn_WordMapping.num_word\n\ncn_model = BiLSTM_CRF.BiLSTMCRF(cn_tag_size, cn_vocab_size, dropout, embed_size, hidden_size).to(device)\nfor name, param in cn_model.named_parameters():\n    if 'weight' in name:\n        nn.init.normal_(param.data, 0, 0.01)\n    else:\n        nn.init.constant_(param.data, 0)\noptimizer = torch.optim.Adam(cn_model.parameters(), lr=lr)\nfor epoch in range(max_epoch):\n    num_iter = 0\n    for sentences, tags in tqdm(batch_iter(cn_train_data, batch_size=batch_size)):\n        num_iter += 1\n        sentences, sent_lengths = pad(sentences, cn_vocab_size - 1, device)\n        tags, _ = pad(tags, cn_tag_size - 1, device)\n\n        optimizer.zero_grad()\n        batch_loss = cn_model(sentences, tags, sent_lengths)\n        loss = batch_loss.mean()\n        loss.backward()\n        torch.nn.utils.clip_grad_norm_(cn_model.parameters(), max_norm=clip_max_norm)\n        optimizer.step()\n\n    print('Epoch: %d,

In [7]:
""" # the training of transformer
from my_transformer import Transformer

dim_embed = 100
max_len = 300
num_heads = 5
dim_feedforward = 400
dropout = 0
num_encoder = 6

lr = 0.001
max_epoch = 30
batch_size = 32
loss_fn = nn.CrossEntropyLoss()

# Chinese version
cn_model = Transformer(
    dim_embed,
    cn_vocab_size,
    max_len,
    num_encoder,
    num_heads,
    dim_feedforward,
    cn_tag_size,
    dropout)
cn_model.to(device)
optimizer = torch.optim.Adam(cn_model.parameters(), lr=lr)
for epoch in range(max_epoch):
    loss_sum = 0
    num = 0
    for sentences, tags in tqdm(batch_iter(cn_train_data, batch_size=batch_size)):
        sentences, sent_lengths = pad(sentences, cn_vocab_size - 1, device)
        tags, _ = pad(tags, cn_tag_size - 1, device)

        mask_sentences = torch.ones_like(sentences)
        mask_sentences[sentences == cn_vocab_size - 1] = 0
        mask_tags = torch.ones_like(tags)
        mask_tags[tags == cn_tag_size - 1] = 0

        optimizer.zero_grad()
        
        output = cn_model(sentences, mask_sentences)
        output = output.view(-1, output.shape[-1])
        tags = tags.view(-1)
        loss = loss_fn(output, tags)
        loss_sum += loss.item()
        num += 1
        loss.backward()
        optimizer.step()
    print(f"epoch: {epoch}, loss: {loss_sum / num}")

cn_model_save_path = "transformer/Chinese.pkl"
with open(cn_model_save_path, "wb") as f:
    pickle.dump(cn_model, f)


#English version
en_model = Transformer(
    dim_embed,
    en_vocab_size,
    max_len,
    num_encoder,
    num_heads,
    dim_feedforward,
    en_tag_size,
    dropout)
en_model.to(device)
optimizer = torch.optim.Adam(en_model.parameters(), lr=lr)
for epoch in range(max_epoch):
    loss_sum = 0
    num = 0
    for sentences, tags in tqdm(batch_iter(en_train_data, batch_size=batch_size)):
        sentences, sent_lengths = pad(sentences, en_vocab_size - 1, device)
        tags, _ = pad(tags, en_tag_size - 1, device)

        mask_sentences = torch.ones_like(sentences)
        mask_sentences[sentences == en_vocab_size - 1] = 0
        mask_tags = torch.ones_like(tags)
        mask_tags[tags == en_tag_size - 1] = 0

        optimizer.zero_grad()
        
        output = en_model(sentences, mask_sentences)
        output = output.view(-1, output.shape[-1])
        tags = tags.view(-1)
        loss = loss_fn(output, tags)
        loss_sum += loss.item()
        num += 1
        loss.backward()
        optimizer.step()
    print(f"epoch: {epoch}, loss: {loss_sum / num}")

en_model_save_path = "transformer/English.pkl"
with open(en_model_save_path, "wb") as f:
    pickle.dump(en_model, f) """

' # the training of transformer\nfrom my_transformer import Transformer\n\ndim_embed = 100\nmax_len = 300\nnum_heads = 5\ndim_feedforward = 400\ndropout = 0\nnum_encoder = 6\n\nlr = 0.001\nmax_epoch = 30\nbatch_size = 32\nloss_fn = nn.CrossEntropyLoss()\n\n# Chinese version\ncn_model = Transformer(\n    dim_embed,\n    cn_vocab_size,\n    max_len,\n    num_encoder,\n    num_heads,\n    dim_feedforward,\n    cn_tag_size,\n    dropout)\ncn_model.to(device)\noptimizer = torch.optim.Adam(cn_model.parameters(), lr=lr)\nfor epoch in range(max_epoch):\n    loss_sum = 0\n    num = 0\n    for sentences, tags in tqdm(batch_iter(cn_train_data, batch_size=batch_size)):\n        sentences, sent_lengths = pad(sentences, cn_vocab_size - 1, device)\n        tags, _ = pad(tags, cn_tag_size - 1, device)\n\n        mask_sentences = torch.ones_like(sentences)\n        mask_sentences[sentences == cn_vocab_size - 1] = 0\n        mask_tags = torch.ones_like(tags)\n        mask_tags[tags == cn_tag_size - 1] =